## Bibliotecas
Antes de prosseguir, vamos importar as bibliotecas necessárias para nossa análise:

In [17]:
# Manipulação de dados
import pandas as pd

# Requisições de APIs
import requests

# Conexão com Google Cloud, Big Query
from pandas_gbq import to_gbq
from google.oauth2 import service_account

## Requisição de API
Vamos iniciar importando a base de dados da Fake Store API.

Para a análise que faremos, precisamos apenas dos endpoints do carrinho (carts) e dos produtos (products).

In [18]:
def consumir_dados(endpoint):
    """
    Função para consumir dados da API Fake Store.

    Parâmetro:
        endpoint : O endpoint da API a ser acessado (ex: "products" ou "carts").

    Retorna: Os dados retornados pela API em formato JSON.
    """
    url_base = "https://fakestoreapi.com"
    url = f"{url_base}/{endpoint}"
    
    try:
        response = requests.get(url)
        dados = response.json()  # Converte a resposta para JSON
        return dados
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API: {e}")

In [19]:
# Consumindo os dados dos carrinhos (carts)
df_carrinhos = pd.DataFrame(consumir_dados('carts'))

# Consumindo os dados dos produtos (products)
df_produtos = pd.DataFrame(consumir_dados('products'))

## Tratamento de bases
Vamos tratar e unificar os dados para termos um dataframe limpo e organizado, seguindo os critérios estabelecidos:

In [20]:
# No dataframe de carrinhos, transformando a coluna de lista em várias linhas
df_carrinhos = df_carrinhos.explode('products').reset_index(drop=True)

# Normalizando a coluna 'products', separando-a em duas colunas
df_carrinhos_normalize = pd.json_normalize(df_carrinhos['products'])

# Concatenando os dados normalizados com o dataFrame original
df_carrinhos = pd.concat([df_carrinhos.drop(columns='products'), df_carrinhos_normalize], axis=1)


In [21]:
# Criando base de data mais recente  em que o usuário adicionou produtos ao carrinho
df_mais_recente = df_carrinhos.groupby('userId')['date'].max().reset_index()

# Juntando bases de carrinhos e produtos
df_joined = pd.merge(df_carrinhos, df_produtos, how='left', left_on='productId', right_on='id')

# Agrupando e identificando a categoria em que cada usuário tem mais produtos adicionados ao carrinho
df = (df_joined.groupby(['userId', 'category'])['quantity']
      .sum()
      .reset_index()
      .loc[lambda x: x.groupby('userId')['quantity'].idxmax()]
      .drop(columns=['quantity'])
      )

# Juntando base com data mais recente
df = pd.merge(df, df_mais_recente, how='left', on='userId')

# Dando nomes mais significativos às colunas
df = df.rename(columns={
    'date': 'data_ultima_adicao',
    'category': 'categoria_mais_adicionada'
})[['userId', 'data_ultima_adicao', 'categoria_mais_adicionada']]

## Visualizando base final
A base final deve conter as seguintes informações:

- identificador de usuário

- data mais recente em que o usuário adicionou produtos ao carrinho

- categoria em que o usuário tem mais produtos adicionados ao carrinho

Como podemos observar a seguir, todos os critérios foram cumpridos.

In [22]:
display(df)

,userId,data_ultima_adicao,categoria_mais_adicionada
0,1,2020-03-02T00:00:00.000Z,men's clothing
1,2,2020-03-01T00:00:00.000Z,men's clothing
2,3,2020-03-01T00:00:00.000Z,men's clothing
3,4,2020-03-01T00:00:00.000Z,electronics
4,8,2020-03-01T00:00:00.000Z,women's clothing


## Carregando base final
Vamos carregar a base final para o BigQuery, bem como salvar a base em um CSV local


In [ ]:
# Salvando em CSV
df.to_csv('../data/arquivo_tratado.csv', index=False)

# Carregando credenciais de conta google
credentials = service_account.Credentials.from_service_account_file(
    'GBQ.json',
)

# Parâmetros para o carregamento
project_id = 'optical-realm-435117-k3' # Altere para o nome do seu projeto
destination_table = 'bigquery_test.user_cart'

# Carrega o DataFrame no BigQuery usando pandas_gbq.to_gbq
to_gbq(df,
       destination_table,
       project_id = project_id,
       if_exists = 'replace',
       credentials = credentials)